In [1]:
import re
import numpy as np
import pandas as pd
import json

In [2]:
from libcg.util.commentutil import CommentUtil
from libcg.util.fileutil import FileUtil
from libcg.util.treeutil import TreeUtil

In [3]:
def runMainSetup(cfgfile, sheetName, futil, tutil):
    cfg = pd.ExcelFile(cfgfile)
    df =  pd.read_excel(cfg, sheetName)
    tutil.getVar()
    for index, row in df.iterrows():
        tutil.getTemplate(row['CodeBlockID'])
    for index, row in df.iterrows():
        tutil.prepBlock(tutil.getReservedToken('BlockName',row), tutil.getReservedToken('CodeBlockID',row), {
        })
    for filename, blocks in tutil.trees['BLKTREE'].items():
        tutil.injectCode(filename, blocks)

In [4]:
COMMENT_FUNCT={
    'CS': CommentUtil.commentSlash,
    'TS': CommentUtil.commentSlash,
    'HTML': CommentUtil.commentHTML,
    'SAS': CommentUtil.commentSAS
}

In [5]:
RULES = {
    'UNIQ' : {
        0:{
        }
    },
    'STARTMARKER':".*GENCODE:MARKER:.*:START.*",
    'ENDMARKER':".*GENCODE:MARKER:.*:END.*",
    'LEVEL_1_SUB':'##BLOCK##',
    'LEVEL_2_SUB':{
        'VariableID':{
            '##(RESOLVEVAR)\(([^#]+)\)##': TreeUtil.funcResolveVar,
            '##(PRESERVE)\(([^#]+)\)##': TreeUtil.funcResolvePreserve
        }
    },
    'LEVEL_3_SUB':{
        'Param': {
            '(!!!PARAM:[^:]+:!!!)': TreeUtil.funcResolveParam,
            '(!!!PARAMDQUOTE:[^:]+:!!!)': TreeUtil.funcResolveParamDQuote,
            '(!!!PARAMDQUOTECOMMA:[^:]+:!!!)': TreeUtil.funcResolveParamDQuoteComma,
            '(!!!PARAMCOMMA:[^:]+:!!!)': TreeUtil.funcResolveParamComma
        }
    },
    'LEVEL_3_RULES':{
        "Replace": {
            "FieldDelimiter": TreeUtil.level3_rep_fd
        }
    },
    'ESCAPERULE':{
        'SQL': TreeUtil.padQuote
    },
    # NULLABLERULE is to cater for C# nullable declaration for datatype, adds a question mark at the end
    'NULLABLERULE':[
        'double',
        'int'
    ],
    'CONSTANTFUNC':{
        '!!!COMMA!!!': TreeUtil.funcResolveComma
    }
}

In [6]:
TREES = {
	'TPLTREE' : {},
	'BLKTREE' : {},
	'PHYTREE' : {},
	'DATTREE' : {},
	'CLRTREE' : {},
	'VARTREE' : {},
	'NVPTREE' : {},
	'uniqBlock' : {}
}

In [7]:
configFile="Config/genCodeCfg_Sample01.xls"

In [8]:
futil = FileUtil(configFile, COMMENT_FUNCT)

In [9]:
tutil = TreeUtil(futil, TREES, RULES)

In [10]:
runMainSetup(configFile, 'MainSetup', futil, tutil)